In [ ]:
import numpy as np
import scipy
import scipy.sparse

In [ ]:
# SBM on 2 blocks of size n
# p - inter-block probability of edge
# q - intra-block probability of edge
def SBM(n, p, q):
    I = []
    J = []
    V = []
    # block 1 edges
    for i in range(n):
        for j in range(i):
            if np.random.rand() < p:
                # i-j
                I.append(i)
                J.append(j)
                V.append(1.)
                # j-i
                I.append(j)
                J.append(i)
                V.append(1.)
    
    # block 2 edges
    for i in range(n, 2*n):
        for j in range(n, i):
            if np.random.rand() < p:
                # i-j
                I.append(i)
                J.append(j)
                V.append(1.)
                # j-i
                I.append(j)
                J.append(i)
                V.append(1.)
                
    # intra-block edges
    for i in range(n):
        for j in range(n, 2*n):
            if np.random.rand() < q:
                # i-j
                I.append(i)
                J.append(j)
                V.append(1.)
                # j-i
                I.append(j)
                J.append(i)
                V.append(1.)
                
    return I, J, V
    
I, J, V = SBM(1000, 0.05, 0.01)

In [ ]:
X = np.array([I, J, V]).T

In [ ]:
# np.savetxt("sbm.csv", X, delimiter=',')
np.savetxt("sbm.csv", X, fmt="%d, %d, %f")

In [ ]:
def read_coo(fname):
    Y = np.loadtxt(fname, delimiter=',')
    I = np.array(Y[:,0], int)
    J = np.array(Y[:,1], int)
    V = Y[:,2]
    return scipy.sparse.coo_matrix((np.array(V), (I, J)))

A = read_coo("sbm.csv")

In [ ]:
class sparse_rank1(object):
    def __init__(self, S, alpha, u, v):
        self.S = S
        self.alpha = alpha
        self.u = u
        self.v = v
        self.shape = S.shape
        
    def dot(self, x):
        return self.S.dot(x) + self.alpha*self.u*self.v.dot(x)

In [ ]:


# compute power method
# tol is a key-word argument for convergence tolerance
def power_method(A, tol=1e-8):
    
    # rayleigh quotient
    # returns v^T*Av
    def rq(v, A):
        return v.dot(A.dot(v))
    
    n = A.shape[1]
    # generate random vector with unit length
    v = np.random.normal(0, 1, n)
    v /= np.linalg.norm(v)
    
    rqs = [] # keep track of rayleigh quotients as we progress
    rqs.append(rq(v, A))
    converged = False
    
    while True:
        
        # v <- A*v
        v = A.dot(v)
        # normalize v
        v /= np.linalg.norm(v)
        
        rqs.append(rq(v,A))
        # check if rayleigh quotient has converged
        if np.abs(rqs[-1] - rqs[-2]) < tol:
            break
    
    # set eigenvalue
    lam = rqs[-1]
    
    return v, lam

In [ ]:
v, lam = power_method(A)

B = sparse_rank1(A, -lam, v, v)
v2, lam2 = power_method(B)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(v, v2)
plt.savefig('sbm.png')
plt.show()